In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.linalg as la

# Cálculo da Matriz de Transferência

A matriz de acoplamento A é consequência do arranjo espacial dos núcleos da fibra. Cada arranjo possui a sua matriz. Então, o nosso primeiro passo é calcular a numericamente algumas matrizes de transferência.

1) Dada as matrizes abaixo, faça a decomposição delas em matrizes de autovalores e autovetores, ou seja, a forma $A = PDP^{-1}$

2)  calcule a $E = exp(D)$ (essa matriz resulta numa matriz tambéem diagonal, onde os elementos na diagonal são a exponencial de cada autovalor);

3) Por fim, calculamos a matriz de transferência $T = PEP^{-1}$

**OBS: P é a matriz de autovetores transposta da matriz A e D é uma matriz diagonal com os autovalores de A;**

### Cálculo de autovalores e autovetores:

A função ``la.eig(square_matrix)`` retorna um vetor de autovalores de A e uma matriz de autovetores da mesma matriz.

Os valores devem ser lidos da seguinte forma: o i-ésimo autovalor do vetor retornado se associa à i-ésima coluna da matriz de autovetores da seguinte forma: 
$$A\lambda = A\vec v$$
Na qual $\lambda$ é o autovalor analisado e $\vec v$ é o autovetor associado, A é a matriz de arranjo. Ademais, os autovetores de A estão sempre localizados nas colunas da matriz de autovetores retornada pela função ``eig`` da lib ``la``.

In [2]:
# A precisa ser uma matriz quadrada!
A = np.array([[4, 1, 0], [0, 2, 1], [0, 0, 3]])
eigvals, eigvecs = la.eig(A)
print('Os autovalores são: \n\n',eigvals)
print('Os autovetores lido em coluna são: \n\n',eigvecs)

Os autovalores são: 

 [4.+0.j 2.+0.j 3.+0.j]
Os autovetores lido em coluna são: 

 [[ 1.         -0.4472136  -0.57735027]
 [ 0.          0.89442719  0.57735027]
 [ 0.          0.          0.57735027]]


Checando os resultados, podemos verificar se $A\lambda_{0} == \lambda_{0}\vec v_{0}$, sendo o primeiro produto uma multiplicação matricial e o segundo um produto escalar.

O exemplo segue apenas para o primeiro autovalor e a primeira coluna da matriz de autovetores, mas pode ser estendida para os outros valores também.

In [3]:
lambda0 = eigvals[0]
eigvec0 = eigvecs[:, 0]

In [4]:
A@eigvec0 == lambda0*eigvec0

array([ True,  True,  True])

## Arranjo Triangular

Entradas: $\alpha, \beta, \gamma$

A = ArranjoTriangular($\alpha, \beta, \gamma$)

T = MatrizTransferencia(A)

In [9]:
def ArranjoTriangular(alpha, beta, gama):
    """
    Retorna a matriz triangular A com parâmetros alpha, beta e gamma.
    complex(x, y) retorna um número complexo da forma x + yi, onde i é o número imaginário
    
    Parâmetros
    --------------------------------------
    :params alpha, beta, gama: alpha é um número real
    
    Retorno
    --------------------------------------
    :return: retorna a matriz arranjo triangular como numpy array cujos valores estão de acordo com os parâmetros recebidos
    """
    A = np.array([[0, complex(0, gama), complex(0, beta)], 
                  [complex(0, gama), 0, complex(0, alpha)], 
                  [complex(0, beta), complex(0, alpha), 0]])
    return A

In [10]:
def MatrizTransferenciaTriangular(A):
    """
    Retorna a matriz de transferência triangular T a partir de uma matriz de arranjo triangular A a partir dos passos já citados
    Erro percentual de 10^(-14) a 10^(-16), ínfimo.
    
    Parâmetros
    --------------------------------------
    :param A: matriz numpy array de transferência triangular T calculada com erro ínfimo.
    """
    eigvals, eigvecs = la.eig(A)
    D = np.array([[eigvals[0], 0, 0], 
                  [0, eigvals[1], 0], 
                  [0, 0, eigvals[2]]])
    P = eigvecs.copy()
    E = math.e**D
    
    # T é a matriz de transferência
    T = P @ E @ la.inv(P)
    print(f'Erro: \n\n{(E - la.inv(P) @ T @ P) * 100/E}')
    
    return T

In [11]:
A = ArranjoTriangular(1, 2, 3)
T = MatrizTransferenciaTriangular(A)
print("\n\nMatriz de transferência: \n\n ", T)

Erro: 

[[ 2.16921827e-14-1.20725636e-14j  0.00000000e+00-2.77555756e-15j
   1.11022302e-14+5.55111512e-15j]
 [ 0.00000000e+00-6.33174069e-15j  1.11238686e-14-2.33566757e-17j
   0.00000000e+00-3.46944695e-16j]
 [ 1.11022302e-14+0.00000000e+00j  2.22044605e-14+5.55111512e-15j
  -2.23804972e-14-1.07429477e-14j]]


Matriz de transferência: 

  [[-0.14870738-0.33599426j -0.4225552 -0.3998916j   1.29009591-0.18361063j]
 [-0.4225552 -0.3998916j  -1.10731104-0.48536506j -1.15369545+0.13225087j]
 [ 1.29009591-0.18361063j -1.15369545+0.13225087j  0.30658844-0.73431637j]]


## Análise do erro

Analisar o erro é importante para verificarmos o quão representativo é a matriz de trasnferência calculada, visto que quando trabalhamos com cálculos realizados computacionalmente, muitas vezes temos truncamento de valores, o que pode levar à propagação de erros e um possível resultado destoante do resultado real.

Portanto, faremos o seguinte: temos a matriz exponencial $E$ calculada em nossa função matriz transferência triangular e calcularemos uma matriz $E_{obt}$ cujo cálculo será feito a partir da matriz de transferência. Após isso, calcularemos a diferença relativa entre as duas utilizando a fórmula: $erro = \frac{E - E_{obt}}{E} \bullet 100$.

Agora, como chegamos à fórmula de $E_{obt}?$ Façamos um pouco de manipulação de matrizes a partir da fórmula de $T$:

$T = PEP^{-1} \Rightarrow P^{-1}T = EP^{-1} \Rightarrow P^{-1}TP = E \Rightarrow E_{obt} = P^{-1}TP$


Realizando um pouco de manipulação de álgebra linear com a fórmula da matriz de transferência, chegamos à fórmula:
$E_{obt} = P^{-1}TP$

Verificando o erro com a fórmula $(\frac{E - P^{-1}TP}{E}) \bullet 100$, temos valores de erros na ordem de $10^{-14}$ a $10^{-16}$, tanto para a parte real quanto para a parte imaginária de $T$. Para isto, basta inserir o código: 

``print((E - la.inv(P) @ T @ P) * 100/E)`` 

na função de cálculo da matriz de transferência.